# Abdullatif Dalab - ID:27880960
# Comp 472 Project - Double Card
# --------------------------------------------------------------------------------------------------------------


# Card Class  
## The card class represents the card object.
## The class has 12 variables that represent: primary position, orientation, card number, secondary position, 
## and other color variables that depend on the orientation variable of the card.
## The class has 3 functions: The constructor, python __repr__ built-in function, and the card_config function.

In [ ]:
#### DEPENDENCIES ####
import numpy as np
import copy
import math as m
import matplotlib.pyplot as plt

#### CARD CLASS ####

class Card:
    
    def __init__(self,primary_position,orientation,card_number):
        self.primary_position = primary_position # Card object primary entry position
        self.orientation = orientation # Game orientations (1-8)
        self.card_number = "" #Serves as a counter
        self.secondary_position = [None,None] # Can be thought of an extension of the object stationed based on card orientation
        self.color_left = ""
        self.color_right = ""
        self.color_up = ""
        self.color_down = ""
        self.dot_left = ""
        self.dot_right = ""
        self.dot_up = ""
        self.dot_down = ""
        
        
    def __repr__(self):
         return "card"+str(self.card_number) # Returns the name of the object
        
    "Given the cards orientation, the color of the card will be configured"
    def card_config(self): 
        
        if self.orientation == 1:
            self.color_left = "R"
            self.color_right = "W"
            self.dot_left = "BD"
            self.dot_right = "WD"
            
        if self.orientation == 2:
            self.color_up = "R"
            self.color_down = "W"
            self.dot_up = "BD"
            self.dot_down = "WD"
            
        if self.orientation == 3:
            self.color_left = "W"
            self.color_right = "R"
            self.dot_left = "WD"
            self.dot_right = "BD"
            
        if self.orientation == 4:
            self.color_up = "W"
            self.color_down = "R"
            self.dot_up = "WD"
            self.dot_down = "BD"
            
        if self.orientation == 5:
            self.color_left = "R"
            self.color_right = "W"
            self.dot_left = "WD"
            self.dot_right = "BD"
            
        if self.orientation == 6:
            self.color_up = "R"
            self.color_down = "W"
            self.dot_up = "WD"
            self.dot_down = "BD"
            
        if self.orientation == 7:
            self.color_left = "W"
            self.color_right = "R"
            self.dot_left = "BD"
            self.dot_right = "WD"
            
        if self.orientation == 8:
            self.color_up = "W"
            self.color_down = "R"
            self.dot_up = "BD"
            self.dot_down = "WD"
       
    

# Board class
## The board class represents the board of the game. Card objects are inserted
## into the board.
## The class has 8 variables: total cards, player1_cards, player2_cards,recycled_card, game_counter,  recycle_ ounter, last_action and a X,  a 12x8 numpy matrix.
## The class has 12 functions: The constructor, col_mappings, row_mappings, insert, insert_rec, plot_map, 
## create_color_string_matrix, create_eval_matrix, check_row, check_col, and check_diag, and check_for_seq

In [ ]:
#### Board class ####

class Board():
    
    "Board configuration"
    
    def __init__(self):
        self.total_cards = 24
        self.recycle_counter = 16
        self.recycled_card = None
        self.player1_cards = 12
        self.player2_cards = 12
        self.game_counter = 0
        self.X = np.empty((12,8),dtype=object)
        self.last_action = None # Last insert that happened on the board.
        
        
    "Map from custom input to numpy array input"
    
    def col_mappings(self):
        return {"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7} 
    
    def row_mappings(self):
        return {1:11,2:10,3:9,4:8,5:7,6:6,7:5,8:4,9:3,10:2,11:1}   
    
    "Insert card object into numpy board"

    def insert(self,X,primary_position,orientation):
        
        global counter
        counter+=1
        
        card = Card(primary_position, orientation,counter) # Creates the card object
        
        if primary_position[0] is not 11: 
            if X[primary_position[0]+1,primary_position[1]] is None: # If position is not on the first row
                print("illegal move")                                #  and the cell below it is empty
                counter-=1
                return "illegal move"
        if X[primary_position[0],primary_position[1]] is not None: # If the insert cell is already occupied
            print("position is taken")
            counter-=1
            return "position is taken"
        else:
            if orientation == 1 or orientation == 3 or orientation == 5 or orientation ==7:
                if primary_position[1] + 1 > 7: # In order not to go out of bounds
                    print("illegal move")
                    counter-=1
                    return "illegal move"
                X[card.primary_position[0],card.primary_position[1]] = card
                card.secondary_position[0] = card.primary_position[0] #row
                card.secondary_position[1] = card.primary_position[1]+1 #col 
                if card.primary_position[0] != 11 and X[card.secondary_position[0]+1,card.secondary_position[1]] is None:
                    print("illegal move")    # This checks for the illegal entry example given in the project outline
                    X[card.primary_position[0],card.primary_position[1]] = None # Remove what's in the primary position given
                    counter-=1                                                  # that there has been an illegal move
                    return "illegal move"
                X[card.secondary_position[0],card.secondary_position[1]] = card # The card object secondary position

            if orientation == 2 or orientation == 4 or orientation == 6 or orientation == 8:

                if  X[card.primary_position[0],card.primary_position[1]] is not None:
                    print("position is taken")
                    counter-=1
                    return "position is taken"
                X[card.primary_position[0],card.primary_position[1]] = card
                card.secondary_position[0] = card.primary_position[0]-1 # position of the second object will be in the row above
                card.secondary_position[1] = card.primary_position[1]
                X[card.secondary_position[0],card.secondary_position[1]] = card
                
            self.last_action = [[primary_position], orientation]
                
        return "no error"
    
    """
       Insert_rec is an insert function used to help bring up the best possible move to the current level in the
       minimax_rec function. _rec refers to the set of functions used in the recycling stage of the AI mode.
    """
    
    def insert_rec(self,X,destionation,primary_position,orientation):
        
        global counter
        counter+=1
        
        card = Card(primary_position, orientation,counter) # Creates the card object
        
        if primary_position[0] is not 11: 
            if X[primary_position[0]+1,primary_position[1]] is None: # If position is not on the first row
                print("illegal move")                                #  and the cell below it is empty
                counter-=1
                return "illegal move"
        if X[primary_position[0],primary_position[1]] is not None: # If the insert cell is already occupied
            print("position is taken")
            counter-=1
            return "position is taken"
        else:
            if orientation == 1 or orientation == 3 or orientation == 5 or orientation ==7:
                if primary_position[1] + 1 > 7: # In order not to go out of bounds
                    print("illegal move")
                    counter-=1
                    return "illegal move"
                X[card.primary_position[0],card.primary_position[1]] = card
                card.secondary_position[0] = card.primary_position[0] #row
                card.secondary_position[1] = card.primary_position[1]+1 #col 
                if card.primary_position[0] != 11 and X[card.secondary_position[0]+1,card.secondary_position[1]] is None:
                    print("illegal move")    # This checks for the illegal entry example given in the project outline
                    X[card.primary_position[0],card.primary_position[1]] = None # Remove what's in the primary position given
                    counter-=1                                                  # that there has been an illegal move
                    return "illegal move"
                X[card.secondary_position[0],card.secondary_position[1]] = card # The card object secondary position

            if orientation == 2 or orientation == 4 or orientation == 6 or orientation == 8:

                if  X[card.primary_position[0],card.primary_position[1]] is not None:
                    print("position is taken")
                    counter-=1
                    return "position is taken"
                X[card.primary_position[0],card.primary_position[1]] = card
                card.secondary_position[0] = card.primary_position[0]-1 # position of the second object will be in the row above
                card.secondary_position[1] = card.primary_position[1]
                X[card.secondary_position[0],card.secondary_position[1]] = card
                
            self.last_action = [destionation,primary_position, orientation]
                
        return "no error"
    
    """ The function plot_map doesn't represent the card blocks accurately and
        is not meant to do so, it acts as a naive visual aid. Given the symmetry, it 
        acts a color map and will visualize consecutive blocks of similar configuaration"""
    
    def plot_map(self,Y):
            M = np.zeros((12,8))
            for i in range(12):
                for j in range(8):
                    if type(Y[i,j]) == str:
                        if (Y[i,j]) == "W" or Y[i,j]=="WD":
                            M[i,j] = 3
                        else:
                            M[i,j] = 1

            from scipy import ndimage
            M1 = ndimage.rotate(M, 180)
            plt.pcolor( M1 , cmap="hot")
            plt.show()

    
    """ create_color_string_matrix takes the matrix(board) of card objects and maps each object to its exact color or dot 
        configuration depending on the boolean value (color). The mappings are stored in matrix Y, which is returned. 
    """
    
    def create_color_string_matrix(self,X,color=True):
        
            Y = np.empty((12,8),dtype=object) # Initialize return matrix
            if color == True:
                rows = 12
                cols = 8
                temp = []
                for i in range(rows):
                    for j in range(cols):
                        c = X[i,j] # store object
                        if type(c) is Card: # if c is a card object
                            c.card_config() # Configure the colors of the card depending on the orientation stored
                            if c.orientation is 1 or  c.orientation is 3 or  c.orientation is 5 or  c.orientation is 7:
                                if [i,j] != c.secondary_position: # In order not to do it again for the other duplicate c object
                                    Y[i,j] = c.color_left 
                                    Y[i,j+1] = c.color_right 
                            if c.orientation is 2 or  c.orientation is 4 or  c.orientation is 6 or  c.orientation is 8:
                                if c.primary_position == [i,j]:
                                    Y[i,j] = c.color_down 
                                elif c.secondary_position == [i,j]:
                                    Y[i,j] = c.color_up 

            else:
                "Map the dot configuration into the string Matrix"
                rows = 12
                cols = 8
                temp = []
                for i in range(rows):
                    for j in range(cols):
                        c = X[i,j] # store object
                        if type(c) is Card: # if c is a card object
                            c.card_config() # Configure the colors of the card depending on the orientation stored
                            if c.orientation is 1 or  c.orientation is 3 or  c.orientation is 5 or  c.orientation is 7:
                                if [i,j] != c.secondary_position: # In order not to do it again for the other duplicate c object
                                    Y[i,j] = (c.dot_left)
                                    Y[i,j+1] = (c.dot_right)
                            if c.orientation is 2 or  c.orientation is 4 or  c.orientation is 6 or  c.orientation is 8:
                                if c.primary_position == [i,j]:
                                    Y[i,j] = (c.dot_down)
                                elif c.secondary_position == [i,j]:
                                    Y[i,j] = (c.dot_up)
        
        
            return Y
        
    """This function generates a string matrix that's used in the evaluate(heurisitic) function in the AI class.
        Given that e(x) requires the color and dot configuration in a cell to perform the end summation, this
        is the matrix that's required as input.
    """   
    def create_eval_matrix(self,X):
            
                Y = np.empty((12,8),dtype=object) # Initialize return matrix
                rows = 12
                cols = 8
                temp = []
                for i in range(rows):
                    for j in range(cols):
                        c = X[i,j] # store object
                        if type(c) is Card: # if c is a card object
                            c.card_config() # Configure the colors of the card depending on the orientation stored
                            if c.orientation is 1 or  c.orientation is 3 or  c.orientation is 5 or  c.orientation is 7:
                                if [i,j] != c.secondary_position: # In order not to do it again for the other duplicate c object
                                    Y[i,j] = c.color_left + "-" + str(c.dot_left)
                                    Y[i,j+1] = c.color_right + "-" + str(c.dot_right)
                            if c.orientation is 2 or  c.orientation is 4 or  c.orientation is 6 or  c.orientation is 8:
                                if c.primary_position == [i,j]:
                                    Y[i,j] = c.color_down + "-" + str(c.dot_down)
                                elif c.secondary_position == [i,j]:
                                    Y[i,j] = c.color_up + "-" + str(c.dot_up)
                return Y
        
        
    """       
        check_for_seq function checks for a sequence of four identical strings in a list.
        For example, given ["white","white","red","white","white","white","white"],
        the function will be able to count the 4 consecutive "white" strings and
        return a string "win" that'll be used to halt the game and declare the winner.
        
        
        check_row ,check_col, and check diag are the three functions that iterate through the matrix to check the winning status.
        They all utilize the check_for_seq function, but the manner with which the matrix is iterated through and the lists
        gathered differs.
        
    """    
        
    def check_row(self,Y):
             
            def check_for_seq(alist):
                counter=0
                temp=""
                string = ""
                for i in range(len(alist)):
                    if type(alist[i]) == str:
                        string = alist[i]
                        if(string != temp):
                            counter=0
                        counter+=1
                        if(counter==4):
                            return "win"
                            break
                    else:
                        counter = 0
                    temp = copy.deepcopy(string)
                return "no win"
            
            rows = 12
            cols = 8
            "Iterate through every row, create a list of it's consecutive string elements, and apply check_for seq to determine status"
            for i in range(rows):
                temp = []
                for j in range(cols):
                        temp.append(Y[i,j])
                result = check_for_seq(temp)  
                if result == "win":
                    return result



            return result

    def check_col(self,Y):
        
            def check_for_seq(alist):
                counter=0
                temp=""
                string = ""
                for i in range(len(alist)):
                    if type(alist[i]) == str:
                        string = alist[i]
                        if(string != temp):
                            counter=0
                        counter+=1
                        if(counter==4):
                            return "win"
                            break
                    else:
                        counter = 0
                    temp = copy.deepcopy(string)
                return "no win"
            
            rows = 12
            cols = 8
            
            "Iterate through every row, create a list of it's consecutive string elements, and apply check_for seq to determine status"
            for j in range(cols):
                temp = []
                for i in range(rows):
                    if type(Y[i,j]) == str:
                        temp.append(Y[i,j])
                result = check_for_seq(temp) 
                if result == "win":
                    return result



            return result



    def check_diag(self,Y):
        
            result="no win"
            def check_for_seq(alist):
                counter=0
                temp=""
                string = ""
                for i in range(len(alist)):
                    if type(alist[i]) == str:
                        string = alist[i]
                        if(string != temp):
                            counter=0
                        counter+=1
                        if(counter==4):
                            return "win"
                            break
                    else:
                        counter = 0
                    temp = copy.deepcopy(string)
                return "no win"
            
            l1 = [] # l1 is going to be a list of lists that contains all the possible diagonals in the matrix
            
            ### 1st diagonal ###
            for k in range(12):
                row = k
                col = 0
                temp1 = []
                while(row >=0 and col < 8):
                    temp1.append(Y[row,col])
                    row -=1
                    col +=1
                l1.append(temp1)


            for j in range(1,8):
                row = 11
                temp1=[]
                while(j <= 7 and row >=0 ):
                    temp1.append(Y[row,j])
                    row-=1
                    j+=1
                l1.append(temp1)

            ### 2nd (mirror/opposite) diagonal ### 
            for i in range(12):
                row = i
                col = 0
                temp2 = []
                while(row<=11 and col <8):
                    temp2.append(Y[row,col])
                    row+=1
                    col+=1
                l1.append(temp2)

            for j in range(1,8):
                row = 0
                col = j
                temp2 = []
                while(col <=7 and row < 11):
                    temp2.append(Y[row,col])
                    row+=1
                    col+=1
                l1.append(temp2)    
            # go through every list in l1 and filter the Nones and empty lists out
            filtered_list= []
            for alist in l1:
                alist = list(filter(None,alist))
                if alist != []:
                    filtered_list.append(alist)
        
            # in every filtered list e, check if there exists a consecutive sequence of similar strings given that e has more than 3 strings
            for e in filtered_list:
                if len(e) >3:
                    result = check_for_seq(e)
                    if result == "win":
                        return result


            return result
        
        
        
    def check_board(self,player):
        
        if player == "color":
            Y = board.create_color_string_matrix(board.X)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        else:
            Y = board.create_color_string_matrix(board.X,False)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured

        return Y,result1,result2,result3

    

# AI class
## This class holds the functions responsible for the correct implementation of
## the minimax algorithm. 
## The class has 10 functions: The constructor, generate_naive_coordinates,
## evaluate, trace, generate_legal_actions, generate_children_states,
## minimax, generate_legal_rec_actions, generate_rec_children_states, and 
## minimax_rec.
### functions with _rec in their names are used in the recycling mode in the AI stage. 




In [ ]:
class AI:
    
    
    def __init__(self):
        self.L2count = [] # stores the evaluations that happend at the last level
        self.L1count = [] # stores the evaluations that happend at first level
        
        
    "This function generates the coordinates (as described in the pdf) required to compute e(x)"
    def generate_naive_coordinates(self):
        Z = np.zeros((12,8))
        n = 1
        row = 11
        while row >= 0:
            col = 7
            while col >= 0:
                Z[row,col] = n
                n+=1
                col-=1
            n+=2
            row-=1        
        Z = np.flip(Z,axis = 1)
        return Z

    "naive-heuristic function"
    def evaluate(self,board):
        import random
        b = Board()
        Y = b.create_eval_matrix(board.X)
        ww = 0
        wb = 0
        rb = 0
        rw = 0
        Z = algo.generate_naive_coordinates()
       
        for row in range(0,12):
            for col in range(0,8):
                if Y[row,col] == "W-WD":
                    ww += Z[row,col]
                elif Y[row,col] == "W-BD":
                    wb += Z[row,col]
                elif Y[row,col] == "R-BD":
                    rb+=Z[row,col]
                elif Y[row,col] == "R-WD":
                    rw +=Z[row,col]


        e = ww + 3*wb - 2*rb - 1.5*rw 
       

        return e
    
    "This traces computes the tracing required."
    def trace(self,file):
        file.write(str(len(algo.L2count)) +"\n")
        if len(algo.L1count) > 0:
            file.write(str(round(algo.L1count[-1],1)) +"\n")
        file.write("\n")
        if len(algo.L1count) > 0:
            [file.write(str(round(e,1))+"\n") for e in algo.L1count]
        file.write("\n")
        self.L1count = []
        self.L2count = []
    
    # Functions for placement state:
    "The legal actions generated from this functions are for the placement mode."
    def generate_legal_actions(self,X):

        possible_actions = []
        odd_orientations = [1,3,5,7]
        even_orientations = [2,4,6,8]

        for row in range(0,12):
            for col in range(0,8):


                if row - 1 >= 0 and col + 1 <= 8 and row == 11:

                    if col < 7:
                        if type(X[row,col]) is not Card and type(X[row,col+1]) is not Card:
                            [possible_actions.append([[row,col], o]) for o in  odd_orientations]

                    if type(X[row,col]) is not Card and type(X[row-1,col]) is not Card:
                        [possible_actions.append([[row,col], o]) for o in  even_orientations]


                if row - 1 >= 0 and col + 1 <= 8 and row != 11:


                    if col < 7:

                        if type(X[row,col]) is not Card and type(X[row,col+1]) is not Card and type(X[row+1,col]) is Card and type(X[row+1,col+1]) is Card:

                            [possible_actions.append([[row,col], o]) for o in  odd_orientations]

                    if type(X[row,col]) is not Card and type(X[row-1,col]) is not Card and type(X[row+1,col]) is Card:

                        [possible_actions.append([[row,col], o]) for o in  even_orientations]

        return possible_actions
    
    "This functions generates all the possible children states given all the legal actions that can taken place in the parent state. "
    "The list of boards returned can be thought of a list of children nodes/states for the parent node/state."
    def generate_children_states(self,board):
        
        possible_states = algo.generate_legal_actions(board.X)
        boards = [Board() for i in range(len(possible_states))]
        i = 0

        for state in possible_states:
            boards[i].X = copy.deepcopy(board.X)
            check = boards[i].insert(boards[i].X,state[0],state[1])
           
            i+=1

        return boards
    
    "Minimax implementation"

    def minimax(self,state,depth,maxplayer):
        if depth == 0:
                algo.L2count.append(H.evaluate_max(state)) # Appends all the evaluations at Level 3
                return H.evaluate_max(state), state.last_action

        children = algo.generate_children_states(state) # Generates the children nodes
        action = None
        
        if maxplayer:
            max_eval = -m.inf 
            for child in children:
                e = self.minimax(child,depth-1,False)
                "Store the max evaluation and it's corresponding action. The actions serves as a potential best possible move."
                if e[0] > max_eval:
                    max_eval = e[0]
                    action = child.last_action
                
                algo.L1count.append(e[0]) # Appends all the evaluations at Level2

            return max_eval,action # returns the best possible move and it's evaluation
        else:
            min_eval = m.inf 
            for child in children:
                e = self.minimax(child,depth-1,True)
                if e[0] < min_eval:
                    min_eval = e[0]
                    action = e[1]

            return min_eval,action

    #### Functions for recycling mode
        
    "This function generates all the legal moves that can take place during the recycling mode"
    
    def generate_legal_rec_actions(self,board):
        
        "Used to filter duplicate moves in case there has been some overlapping"
        def filter_list(alist):
            alist = set([tuple(x) for x in alist])
            alist = [list(x) for x in alist]
            alist.sort(key=lambda x: [x[0],x[1]])  
            return alist
    
        possible_actions = []
        odd = [1,3,5,7]
        even = [2,4,6,8]
        mcards = [] # A list that holds movable cards
        
        # Find movable cards/ cards that can be legally moved:

        for row in range(12):
            for col in range(8):
                t_card = board.X[row,col]
                if type(t_card) is Card:
                    if t_card.primary_position == [row,col]:
                         if t_card != board.recycled_card:
                            if t_card.orientation in odd:
                                if type(board.X[row-1,col]) is not Card and type(board.X[row-1,col+1]) is not Card:
                                    mcards.append(board.X[row,col])
                            elif board.X[row,col].orientation in even:
                                if t_card.secondary_position[0]-1 == -1:
                                    mcards.append(board.X[row,col])
                                if type(board.X[t_card.secondary_position[0]-1,t_card.secondary_position[1]]) is not Card:
                                    mcards.append(board.X[row,col])


        "Generate possible recycling moves in fixed positions"
        for card in mcards:
            r1 = card.primary_position[0]
            c1 = card.primary_position[1]

            if card.orientation in odd:
                [possible_actions.append([r1,c1,o,r1,c1])for o in odd if o != card.orientation]
                [possible_actions.append([r1,c1,o,r1,c1]) for o in even]
                [possible_actions.append([r1,c1,o,r1,c1+1]) for o in even]
            elif card.orientation in even:
                [possible_actions.append([r1,c1,o,r1,c1]) for o in even if o != card.orientation]


        # possible combinations of moves across the board
        
        for i in range(len(mcards)):
            t_card = mcards[i]
            for j in range(len(mcards)):
                if j != i:

                    r1 = t_card.primary_position[0]
                    c1 = t_card.primary_position[1]
                    r2 = mcards[j].primary_position[0]
                    c2 = mcards[j].primary_position[1]

                    # MOVING ODD-TO-ODD
                    if t_card.orientation in odd:
                        if mcards[j].orientation in odd:

                            [possible_actions.append([r1,c1,o,r2-1,c2]) for o in odd]
                            [possible_actions.append([r1,c1,o,r2-1,c2]) for o in even]
                            [possible_actions.append([r1,c1,o,r2-1,c2+1]) for o in even]

                            if c2+2 < 8 and c2 - 1 > 0 and r2 - 1 > 0:

                                if type(board.X[r2,c2+2]) is Card and  type(board.X[r2-1,c2+2]) is not Card:
                                    if [r2,c2+2] != [r1,c1]:
                                        [possible_actions.append([r1,c1,o,r2-1,c2+1]) for o in odd]

                                if type(board.X[r2,c2-1]) is Card and  type(board.X[r2-1,c2-1]) is not Card:
                                    if [r2,c2-1] != [r1,c1+1]:
                                        [possible_actions.append([r1,c1,o,r2-1,c2-1]) for o in odd]

                    # MOVING ODD-TO-EVEN   
                    
                    if t_card.orientation in odd:
                        if mcards[j].orientation in even:
                            if type(board.X[r2-2,c2]) is not Card:
                                    [possible_actions.append([r1,c1,o,r2-2,c2]) for o in even]

                            if r2-2 > 0 and c2+1 < 8 and c2-1>0:
                                    if type(board.X[r2-1,c2+1]) is Card and type(board.X[r2-2,c2+1]) is not Card:
                                        [possible_actions.append([r1,c1,o,r2-2,c2]) for o in odd]

                                    if type(board.X[r2-1,c2-1]) is Card and type(board.X[r2-2,c2-1]) is not Card:
                                        [possible_actions.append([r1,c1,o,r2-2,c2-1]) for o in odd]
                                        [possible_actions.append([r1,c1,o,r2-2,c2-1]) for o in even]

                    # MOVING EVEN-TO-ODD
                    
                    if t_card.orientation in even:
                        if mcards[j].orientation in odd:

                            if type(board.X[r2-1,c2]) is not Card:
                                 [possible_actions.append([r1,c1,o,r2-1,c2]) for o in even]
                                 if type(board.X[r2-1,c2+1]) is not Card:
                                     [possible_actions.append([r1,c1,o,r2-1,c2]) for o in odd]
                                     [possible_actions.append([r1,c1,o,r2-1,c2+1]) for o in even]
                                     if c2+2 < 8 and r2 - 1 > 0:
                                         if type(board.X[r2,c2+2]) is Card and type(board.X[r2-1,c2+2]) is not Card:
                                             [possible_actions.append([r1,c1,o,r2-1,c2+1]) for o in odd]

                    # MOVING EVEN-TO-EVEN   
                    
                    if t_card.orientation in even:
                        if mcards[j].orientation in even:
                            if type(board.X[r2-2,c2]) is not Card:
                                [possible_actions.append([r1,c1,o,r2-2,c2]) for o in even]

                    # MOVING ODD-AND-EVEN-TO-LASTROW-IF-EMPTY
                    
                    for col in range(8):
                        if type(board.X[11,col]) is not Card:
                            [possible_actions.append([r1,c1,o,11,col]) for o in even]
                            if col + 1 < 8:
                                if type(board.X[11,col+1]) is not Card:
                                    [possible_actions.append([r1,c1,o,11,col]) for o in odd]



        return filter_list(possible_actions)
    
    "Given the possible legal actions in the recycling stage, this function creates a list of states that are possible to reach"
    "through this actions."
    
    def generate_rec_children_states(self,board):
        
        odd = [1,3,5,7]
        even = [2,4,6,8]
        possible_states = algo.generate_legal_rec_actions(board)
        boards = [Board() for i in range(len(possible_states))]
        
        i = 0

        for state in possible_states:
            boards[i].X = copy.deepcopy(board.X)
            "Remove selected card from the board"
            if boards[i].X[state[0],state[1]].orientation in odd:
                boards[i].X[state[0],state[1]] = None
                boards[i].X[state[0],state[1]+1] = None
            elif  boards[i].X[state[0],state[1]].orientation in even:
                boards[i].X[state[0],state[1]] = None
                boards[i].X[state[0]-1,state[1]] = None
                
            check = boards[i].insert_rec(boards[i].X,[state[0],state[1]],[state[3],state[4]],state[2])
           
            i+=1

        return boards
        
    
    "Minimax implementation with modified code to suit the recycling mode"
    def minimax_rec(self,state,depth,maxplayer):
        
        if depth == 0:
            algo.L2count.append(H.evaluate_max(state))
            return H.evaluate_max(state), state.last_action

        children = algo.generate_rec_children_states(state)
        action = None
        if maxplayer:
            max_eval = -m.inf 
            for child in children:
                e = self.minimax(child,depth-1,False)
                if e[0] > max_eval:
                    max_eval = e[0]
                    action = child.last_action
                    
                algo.L1count.append(e[0])


            return max_eval,action
        else:
            min_eval = m.inf 
            for child in children:
                e = self.minimax(child,depth-1,True)
                if e[0] < min_eval:
                    min_eval = e[0]
                    action = e[1]

            return min_eval,action


  
        
        

# Informed Heuristic class
## This class holds the functions responsible for the implementation of the heuristic function
## The class has 10 functions: Consturctor, four_seq, three_seq, two_seq, check_seq_no_diag, check_seq, calculate_proximity, calculate_cavity, evaluate, evaluate_max.
 


In [ ]:
class Informed_Heuristic:

    def __init__(self):
        self.seq_found = {"4-seq":0,"3-seq":0,"2-seq":0} # Stores the number of 2-3-4 identical segments in a board
        
    "four_seq checks for a sequence of 4 identical segments"    
    def four_seq(self,alist):
            fours = 0
            counter=0
            temp=""
            string=""
            for i in range(len(alist)):
                if type(alist[i]) == str:
                    string = alist[i]
                    if(string != temp):
                        counter=0
                    counter+=1
                    if(counter==4):
                        fours+= 1
                        counter=0
                temp = copy.deepcopy(alist[i])
            
            return fours
    
    "three_seq checks for a sequence of 3 identical segments" 
    def three_seq(self,alist):
            threes = 0
            counter=0
            temp=""
            string=""
            for i in range(len(alist)):
                if type(alist[i]) == str:
                    string = alist[i]
                    if(string != temp):
                        counter=0
                    counter+=1
                    if(counter==3):
                        threes+=1
                        counter=0
                temp = copy.deepcopy(alist[i])
            
            return threes
    "two_seq checks for a sequence of 2 identical segments"     
    def two_seq(self,alist):
            twos = 0
            counter=0
            temp=""
            string=""
            for i in range(len(alist)):
                if type(alist[i]) == str:
                    string = alist[i]
                    if(string != temp):
                        counter=0
                    counter+=1
                    if(counter==2):
                        twos+=1
                        counter=0
                temp = copy.deepcopy(alist[i])
            
            return twos
        

    "check_seq_no_diag checks for all the 4 winning segments in the rows and columns of the board but not the diagonals"
    def check_seq_no_diag(self,Y):
            
            rows = 12
            cols = 8
            for i in range(rows):
                temp = []
                for j in range(cols):
                        temp.append(Y[i,j])
                self.seq_found["4-seq"]+=H.four_seq(temp)
               
            
            
                
            for j in range(cols):
                temp = []
                for i in range(rows):
                        temp.append(Y[i,j])
                self.seq_found["4-seq"]+=H.four_seq(temp)
                
                

        
            return self.seq_found
    "check_seq checks for all 2 and 3 identical segments and 4 winning segments across the rows,columns, and diagonals of the board"
    def check_seq(self,Y):
            
            rows = 12
            cols = 8
            for i in range(rows):
                temp = []
                for j in range(cols):
                        temp.append(Y[i,j])
                self.seq_found["2-seq"]+=H.two_seq(temp)
                self.seq_found["3-seq"]+=H.three_seq(temp)
                self.seq_found["4-seq"]+=H.four_seq(temp)
                
            for j in range(cols):
                temp = []
                for i in range(rows):
                        temp.append(Y[i,j])
                        
                self.seq_found["2-seq"]+=H.two_seq(temp)
                self.seq_found["3-seq"]+=H.three_seq(temp)
                self.seq_found["4-seq"]+=H.four_seq(temp)
                
            l1 = [] # l1 is going to be a list of lists that contains all the possible diagonals in the matrix
            
            ### 1st diagonal ###
            for k in range(12):
                row = k
                col = 0
                temp1 = []
                while(row >=0 and col < 8):
                    temp1.append(Y[row,col])
                    row -=1
                    col +=1
                l1.append(temp1)


            for j in range(1,8):
                row = 11
                temp1=[]
                while(j <= 7 and row >=0 ):
                    temp1.append(Y[row,j])
                    row-=1
                    j+=1
                l1.append(temp1)

            ### 2nd (mirror/opposite) diagonal ### 
            for i in range(12):
                row = i
                col = 0
                temp2 = []
                while(row<=11 and col <8):
                    temp2.append(Y[row,col])
                    row+=1
                    col+=1
                l1.append(temp2)

            for j in range(1,8):
                row = 0
                col = j
                temp2 = []
                while(col <=7 and row < 11):
                    temp2.append(Y[row,col])
                    row+=1
                    col+=1
                l1.append(temp2)    
            # go through every list in l1 and filter the Nones and empty lists out
            
            for l in l1:
                self.seq_found["2-seq"]+=H.two_seq(l)
                self.seq_found["3-seq"]+=H.three_seq(l)
                self.seq_found["4-seq"]+=H.four_seq(l)
 
        
            return self.seq_found
    
    """Calculates the proximity feature of the heuristic. Essentially, it looks for cards of identical configuration
       and increments a proximity variable by 1000 everytime the distance is between the identical cards is 1.
    """
    def calculate_proximity(self,board):
        
        cards = []
        for r in range(12):
            for c in range(8):
                if type(board.X[r,c]) is Card:
                    cards.append(board.X[r,c])

        cards = list(set(cards))

        avg_distance = 0
        proximity = 0           
        for i in range(len(cards)):
            temp = cards[i]
            for j in range(len(cards)):
                if i != j:
                    if cards[i].orientation == cards[j].orientation:
                            dist1 = cards[i].primary_position
                            dist2 = cards[j].primary_position
                            avg_distance = abs(dist1[0] - dist2[0]) + abs(dist1[1] - dist2[1])
                            if avg_distance == 1:
                                proximity += 1000
                              
                            
        return proximity
    "This function checks how many empty spots there is in the last row"
    def calculate_cavity(self,board):
        
        cavity = 0
        for col in range(8):
            if type(board.X[11,col]) is not Card:
                cavity+=1
        if cavity == 0:
            return 1000
        elif cavity == 1:
            return 500
        elif cavity == 2:
            return 250
        else:
            return 0
        
    "Heuristic used in the tournament"
    def evaluate_max(self,board):
        Y = board.create_color_string_matrix(board.X)
        self.check_seq_no_diag(Y)        
        fours = self.seq_found["4-seq"]
        proximity = self.calculate_proximity(board)
        cavity = self.calculate_cavity(board)
        self.seq_found = {"4-seq":0,"3-seq":0,"2-seq":0}
        if fours > 0:
            return 1000000
        e = proximity + cavity
        return e
    
    "Heuristic function used for testing"
    def evaluate(self,board):
        Y = board.create_color_string_matrix(board.X)
        self.check_seq(Y)        
        fours = self.seq_found["4-seq"]
        threes = self.seq_found["3-seq"]
        twos = self.seq_found["2-seq"]
        proximity = self.calculate_proximity(board)
        cavity = self.calculate_cavity(board)
        e = proximity + cavity 
        return e


# Manual testing

In [ ]:
counter = 0
board = Board()
board.create_color_string_matrix(board.X)
t = []

In [ ]:
import time
algo = AI()
H = Informed_Heuristic()
gc = 40

while gc > 0 :
    start_time = time.clock()
    res = algo.minimax(board,2,True)

    board.insert(board.X,res[1][0][0],res[1][1])
    t.append(str(time.clock() - start_time))
    gc-=1
  


In [ ]:
#board.insert(board.X,[11,2],2)
board.create_color_string_matrix(board.X)

# Human Vs Human 

In [ ]:
# Initialize board
board = Board()
col_dict = board.col_mappings() # Map from custom input to numpy input
row_dict = board.row_mappings() # MAo from custom input to numpy input
counter = 0

print("\n input format example: 1 A 1 or Orientation,Column,Row\n")

while board.total_cards != 0:
    
    if board.total_cards == 24:
        player1 = str(input("Player1, please enter what you will play (color or dot)"))
        if player1 == "color":
            player2 = "dot"
        else:
            player2 = "color"
        
    
    
    if board.game_counter % 2 == 0:
        print("Player1: " + "# of Cards left: " + str(board.player1_cards))
        user_input = (input("Enter the position and orientation: ")) # Column of choice
        user_input = user_input.split()
        o = int(user_input[0]) # Orientation
        c = user_input[1]
        r = int(user_input[2])
        if o >= 8 or o <= 0:
            print("Orientations range between 1 - 8, please try again.")
            continue
            
        check = board.insert(board.X,[row_dict[r],col_dict[c]],o) # The dictionaries map the custom input to input that fits the numpy array
        if check == "position is taken" or check == "illegal move": # User will be able to try again (game counter is not affected)
            continue
            
        if player1 == "color":
            Y = board.create_color_string_matrix(board.X)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        else:
            Y = board.create_color_string_matrix(board.X,False)
            result1 = board.check_row(Y) # Check the rows with the dot configuration
            result2 = board.check_col(Y) # Check the cols with the dot configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured

        board.player1_cards-=1
        board.plot_map(Y)
        print(Y)
        if result1 == "win" or result2 == "win" or result3 == "win":
            print("Player 1 has won!")
            break
            
    else:
        print("Player2: " + "# of Cards left = " + str(board.player2_cards))
        user_input = (input("Enter the position and orientation: ")) # Column of choice
        user_input = user_input.split()
        o = int(user_input[0]) # Orientation
        c = user_input[1]
        r = int(user_input[2])
        if o >= 8 or o <= 0:
            print("Orientations range between 1 - 8, please try again.")
            continue
        check = board.insert(board.X,[row_dict[r],col_dict[c]],o)
        if check == "position is taken" or check == "illegal move":
            continue
        if player2 == "color":
            Y = board.create_color_string_matrix(board.X)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        else:
            Y = board.create_color_string_matrix(board.X,False)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        board.player2_cards-=1
        board.plot_map(Y)
        print(Y)
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("Player 2 has won!")
            break
            
        
    board.game_counter+=1 # variable to help alternate between player 1 and 2
    board.total_cards-=1
    
   
   
if(board.total_cards == 0 and result1 == "no win" and result2 == "no win" and result3 == "no win"):
    board.game_counter = 0
    print("")
    print("Recycling mode is on :")
    
    print("\n")
    print("Recycling mode is on :")
    print("\nNote that recycling input follows this format [primary cell you want to move],[orienation of the card],[new cell of the card]")
    print("\n You won't need to specify the second cell of the card you're trying to move because the card orientation already determined it's position.")
    
    while board.recycle_counter > 0:
        
        
        if board.game_counter % 2 == 0:
            
            print("Player1, please insert the details of your recycling move")
            user_input = (input("Enter the position and orientation: ")) # Column of choice
            user_input = user_input.split()
            c1 = user_input[0]
            r1 = int(user_input[1])
            o = int(user_input[2])
            c2 = user_input[3]
            r2 = int(user_input[4])
            
            if o >= 8 or o <= 0:
                print("Orientations range between 1 - 8, please try again.")
                continue
                
            card = board.X[row_dict[r1],col_dict[c1]]
            
            if type(card) == type(None):
                print("There is no card placed here, please try again.")
                continue
                
            if card.orientation == 1 or card.orientation == 3 or card.orientation == 5 or card.orientation == 7:
                if type(board.X[card.primary_position[0]-1,card.primary_position[1]]) is not type(None):
                    print("hole error, please try again")
                    continue
            else:
                if type(board.X[card.secondary_position[0]-1,card.secondary_position[1]]) is not type(None):
                    print("hole error, please try again")
                    continue
                
            if type(board.X[row_dict[r2],col_dict[c2]]) is not type(None) and [r2,c2] != [r1, c1] and card.secondary_position != [row_dict[r2],col_dict[c2]]:
                print("position isn't available for placement, please try again")
                continue
                
            if [r1,c1] == [r2,c2] and card.orientation == o:
                print("You can't not make a move, please try again.")
                continue
            
            
                
            if(board.recycled_card != card):
                board.recycled_card = None
                if [row_dict[r2],col_dict[c2]] == card.secondary_position and o in [2,4,6,8]:
                    board.X[card.primary_position[0],card.primary_position[1]] = None
                    board.X[card.secondary_position[0],card.secondary_position[1]] = None
                    board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    Y = board.create_color_string_matrix(board.X)
                    result1 = board.check_row(Y)
                    result2 = board.check_col(Y)
                    result3 = board.check_diag(Y)
                    card = board.X[row_dict[r2],col_dict[c2]]
                    board.recycled_card = card
                    board.plot_map(Y)
                    print(Y)
                    if result1 == "win" or result2 == "win" or result3 == "win":
                        print("Player 1 has won!")
                        break
                else:
                    check_var = board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    if (check_var != "illegal move" and check_var != "position is taken"):
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        Y = board.create_color_string_matrix(board.X)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                        card = board.X[row_dict[r2],col_dict[c2]]
                        board.recycled_card = card
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break
                    elif check_var != "illegal move" and check_var == "position is taken" and [r2,c2] == [r1, c1]:
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                        Y = board.create_color_string_matrix(board.X)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                        card = board.X[row_dict[r2],col_dict[c2]]
                        board.recycled_card = card
                        board.plot_map(Y)
                        print(Y)
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break
                    else:
                        print("Move is not possible, try again")
                        continue
            else:
                print("This card has been recycled")
                continue
                
        else: # next player
            
            print("Player2, please insert the details of your recycling move")
            user_input = (input("Enter the position and orientation: ")) # Column of choice
            user_input = user_input.split()
            c1 = user_input[0]
            r1 = int(user_input[1])
            o = int(user_input[2])
            c2 = user_input[3]
            r2 = int(user_input[4])
            
            if o >= 9 or o <= 0:
                print("Orientations range between 1 - 8, please try again.")
                continue
            card = board.X[row_dict[r1],col_dict[c1]]
            
            if type(card) == type(None):
                print("There is no card placed here, please try again.")
                continue
                
            if card.orientation == 1 or card.orientation == 3 or card.orientation == 5 or card.orientation == 7:
                if type(board.X[card.primary_position[0]-1,card.primary_position[1]]) is not type(None):
                    print(card.primary_position)
                    print("hole error, please try again")
                    continue
            else:
                if type(board.X[card.secondary_position[0]-1,card.secondary_position[1]]) is not type(None):
                    print(card.secondary_position)
                    print("hole error, please try again")
                    continue
            
                    
            if type(board.X[row_dict[r2],col_dict[c2]]) is not type(None) and [r2,c2] != [r1, c1] and card.secondary_position != [row_dict[r2],col_dict[c2]]:
                print("position isn't available for placement, please try again")
                continue
                
            if [r1,c1] == [r2,c2] and card.orientation == o:
                print("You can't not make a move, please try again.")
                continue
            
            if (card != board.recycled_card):
                board.recycled_card = None
                if [row_dict[r2],col_dict[c2]] == card.secondary_position and o in [2,4,6,8]:
                    board.X[card.primary_position[0],card.primary_position[1]] = None
                    board.X[card.secondary_position[0],card.secondary_position[1]] = None
                    board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    Y = board.create_color_string_matrix(board.X)
                    result1 = board.check_row(Y)
                    result2 = board.check_col(Y)
                    result3 = board.check_diag(Y)
                    card = board.X[row_dict[r2],col_dict[c2]]
                    board.recycled_card = card
                    
                    board.plot_map(Y)
                    print(Y)
                    
                    if result1 == "win" or result2 == "win" or result3 == "win":
                        print("Player 1 has won!")
                        break
                else:
                    check_var = board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    if (check_var != "illegal move" and check_var != "position is taken" ):
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        Y = board.create_color_string_matrix(board.X)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                        card = board.X[row_dict[r2],col_dict[c2]]
                        board.recycled_card = card
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 2 has won!")
                            break
                    elif check_var != "illegal move" and check_var == "position is taken" and [r2,c2] == [r1, c1]:
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                        Y = board.create_color_string_matrix(board.X)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                        card = board.X[row_dict[r2],col_dict[c2]]
                        board.recycled_card = card
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 2 has won!")
                            break
                    else:
                        print("Move is not possible, try again")
                        continue
            else:
                print("This card has been recycled")
                continue
        
        
        board.game_counter+=1
        board.recycle_counter-=1
        
if result1 != "win" and result2 != "win" and result3 != "win":
    print("The game ends in draw.")


# AI vs Human Mode


In [ ]:
# Initialize board
board = Board()
col_dict = board.col_mappings() # Map from custom input to numpy input
row_dict = board.row_mappings() # MAo from custom input to numpy input
counter = 0
algo = AI()
H = Informed_Heuristic()

Trace = str(input("Please enter 'yes' if you'd like to generate a trace file for the minimax function.\n"))
if Trace == "yes":
    file = open("tracemm1.txt","w")

print("\n input format example: 1 A 1 or Orientation,Column,Row\n")
while board.total_cards != 0:
    
    if board.total_cards == 24:
        player1 = str(input("Player1, please enter what you will play (color or dot)"))
        if player1 == "color":
            player2 = "dot"
        else:
            player2 = "color"
        
    
    if board.game_counter % 2 == 0:
        print("Player1: " + "# of Cards left: " + str(board.player1_cards))
        user_input = (input("Enter the position and orientation: ")) # Column of choice
        user_input = user_input.split()
        o = int(user_input[0]) # Orientation
        c = user_input[1]
        r = int(user_input[2])
        if o >= 8 or o <= 0:
            print("Orientations range between 1 - 8, please try again.")
            continue
            
        check = board.insert(board.X,[row_dict[r],col_dict[c]],o) # The dictionaries map the custom input to input that fits the numpy array
        if check == "position is taken" or check == "illegal move": # User will be able to try again (game counter is not affected)
            print(check + ", please try again.")
            continue
            
        if player1 == "color":
            Y = board.create_color_string_matrix(board.X)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        else:
            Y = board.create_color_string_matrix(board.X,False)
            result1 = board.check_row(Y) # Check the rows with the dot configuration
            result2 = board.check_col(Y) # Check the cols with the dot configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured

        board.player1_cards-=1
        
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 == "win" or result3 == "win":
            print("Player 1 has won!")
            break
            
    else:
        
        "res contains the best possible move that was raised to the current level"
        res = algo.minimax(board,2,True)
        
        if Trace == "yes":
            algo.trace(file)
            
        pos = res[1][0][0]
        o = res[1][1]
        
        "insert the best possible move"
        board.insert(board.X,pos,o)
        
        if player2 == "color":
            Y = board.create_color_string_matrix(board.X)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        else:
            Y = board.create_color_string_matrix(board.X,False)
            result1 = board.check_row(Y) # Check the rows with the color configuration
            result2 = board.check_col(Y) # Check the cols with the color configuration
            result3 = board.check_diag(Y) # Y matrix holding the strings is already configured
        board.player2_cards-=1
        
        print("\nAI Boards:\n")
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("Player 2 has won!")
            break

    if board.game_counter == 4:
        board.total_cards = 0
        break
        
    board.game_counter+=1 # variable to help alternate between player 1 and 2
    board.total_cards-=1
    #board.error_count_ai = 0
                  
if Trace == "yes":
    if result1 == "win" or result2 =="win" or result3 == "win":  
        file.close()
        
        
if(board.total_cards == 0 and result1 == "no win" and result2 == "no win" and result3 == "no win"):
    board.game_counter = 0
    print("\n")
    print("Recycling mode is on :")
    print("\nNote that recycling input follows this format [primary cell you want to move],[orienation of the card],[new cell of the card]")
    print("\n You won't need to specify the second cell of the card you're trying to move because the card orientation already determined it's position.")
    
    while board.recycle_counter > 0:
        
        odd = [1,3,5,7]
        even = [2,4,6,8]
        
        if board.game_counter % 2 == 0:
            
            print("Player1, please insert the details of your recycling move")
            user_input = (input("Enter the card position, orienation, and new card positon. EX: A 1 2 D 1 ")) # Column of choice
            user_input = user_input.split()
            c1 = user_input[0]
            r1 = int(user_input[1])
            o = int(user_input[2])
            c2 = user_input[3]
            r2 = int(user_input[4])
            
            if o >= 9 or o <= 0:
                print("Orientations range between 1 - 8, please try again.")
                continue
                
            card = board.X[row_dict[r1],col_dict[c1]]
            
            if type(card) == type(None):
                print("There is no card placed here, please try again.")
                continue
                
            if card.orientation == 1 or card.orientation == 3 or card.orientation == 5 or card.orientation == 7:
                if type(board.X[card.primary_position[0]-1,card.primary_position[1]]) is not type(None):
                    print("hole error, please try again")
                    continue
            else:
                if card.secondary_position[0]-1 > 0:
                    if type(board.X[card.secondary_position[0]-1,card.secondary_position[1]]) is not type(None):
                        print("hole error, please try again")
                        continue
                
            if type(board.X[row_dict[r2],col_dict[c2]]) is not type(None) and [r2,c2] != [r1, c1] and card.secondary_position != [row_dict[r2],col_dict[c2]]:
                print("position isn't available for placement, please try again")
                continue
                
            if [r1,c1] == [r2,c2] and card.orientation == o:
                print("You can't not make a move, please try again.")
                continue
            
            
            "CASE: if the card we want to move is even, and the destionation position overlaps on the secondary position(upper part of the card)"    
            if(card != board.recycled_card):
                board.recycled_card = None
                if [row_dict[r2],col_dict[c2]] == card.secondary_position and o in even:
                    "Empty positions of moved card"
                    board.X[card.primary_position[0],card.primary_position[1]] = None
                    board.X[card.secondary_position[0],card.secondary_position[1]] = None
                    "Insert card with new orientation"
                    board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    "Create the string representation of the matrix"
                    if player1 == "color":
                        Y = board.create_color_string_matrix(board.X)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                    else:
                        "Creates a dot matrix that's checked"
                        Y = board.create_color_string_matrix(board.X,False)
                        result1 = board.check_row(Y)
                        result2 = board.check_col(Y)
                        result3 = board.check_diag(Y)
                        
                    "Have the new card be marked as recycled"
                    board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                    
                    board.plot_map(Y)
                    print(Y)
                    if result1 == "win" or result2 == "win" or result3 == "win":
                        print("Player 1 has won!")
                        break
                else:
                    "CASE: regular check"
                    check_var = board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    if (check_var != "illegal move" and check_var != "position is taken"):
                        "Empty positions of moved card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        
                        if player1 == "color":
                            Y = board.create_color_string_matrix(board.X)
                            result1 = board.check_row(Y)
                            result2 = board.check_col(Y)
                            result3 = board.check_diag(Y)
                        else:
                            Y = board.create_color_string_matrix(board.X,False)
                            result1 = board.check_row(Y)
                            result2 = board.check_col(Y)
                            result3 = board.check_diag(Y)
                            
                        "Have the new card be marked as recycled"    
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break

                    elif check_var != "illegal move" and check_var == "position is taken" and [r2,c2] == [r1, c1]:
                        "Empty positions of moving card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                        if player1 == "color":
                            Y = board.create_color_string_matrix(board.X)
                            result1 = board.check_row(Y)
                            result2 = board.check_col(Y)
                            result3 = board.check_diag(Y)
                        else:
                            Y = board.create_color_string_matrix(board.X,False)
                            result1 = board.check_row(Y)
                            result2 = board.check_col(Y)
                            result3 = board.check_diag(Y)
                            
                        "Have the new card be marked as recycled"
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                       
                        board.plot_map(Y)
                        print(Y)
                        
                        #print(board.recycled_card)
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break
                    else:
                        print("Move is not possible, try again")
                        continue
            else:
                print("This card has been recycled")
                continue
                
        else: # next player
            
            print("\n AI Player \n")
            res = algo.minimax_rec(board,2,True)
            r1 = res[1][0][0] # Row of card to move
            c1 = res[1][0][1] # Col of card to move
            r2 = res[1][1][0] # Row of new destination
            c2 = res[1][1][1] # Col of new destination
            o = res[1][2]     # New orientation
            
            if Trace == "yes":
                algo.trace(file)
                
            "If the card we want to move is (odd or even), remove the following positions"
            if board.X[r1,c1].orientation in odd:
                if c1 + 1 < 8: # Limiting bounds
                    board.X[r1,c1] = None
                    board.X[r1,c1+1] = None
            elif board.X[r1,c1].orientation in even:
                if r1-1 > 0:
                    board.X[r1,c1] = None
                    board.X[r1-1,c1] = None
                    
            "Insert moved card into new location"  
            board.insert(board.X,[r2,c2],o)
            card = board.X[r2,c2]
            "Have the new card marked as recycled"
            board.recycled_card = card
            
            print("\n AI move: " + "[" +str(r1)+ "," +str(c1)+ "]" + "------>" + "[" +str(r2)+ "," +str(c2)+  "]" + " with orientation " + str(o))
            
            if player2 == "color":
                Y = board.create_color_string_matrix(board.X)
                result1 = board.check_row(Y)
                result2 = board.check_col(Y)
                result3 = board.check_diag(Y)
            else:
                Y = board.create_color_string_matrix(board.X,False)
                result1 = board.check_row(Y)
                result2 = board.check_col(Y)
                result3 = board.check_diag(Y)
            
            board.plot_map(Y)
            print(Y)
            
            if result1 == "win" or result2 == "win" or result3 == "win":
                print("Player 2 has won!")
                break
                    
        
        board.game_counter+=1
        board.recycle_counter-=1
        
if result1 != "win" and result2 != "win" and result3 != "win":
    print("The game ends in draw.")

if Trace == "yes":
    file.close()

# Match1 - AI plays as player 1 and chooses configuration

In [ ]:
import random
# Initialize board
board = Board()
col_dict = board.col_mappings() # Map from custom input to numpy input
row_dict = board.row_mappings() # MAo from custom input to numpy input
counter = 0
algo = AI()
H = Informed_Heuristic()

Trace = str(input("Please enter 'yes' if you'd like to generate a trace file for the minimax function.\n"))
if Trace == "yes":
    file = open("tracemm1.txt","w")

print("\n input format example: 1 A 1 or Orientation,Column,Row\n")
while board.total_cards != 0:
    
    if board.total_cards == 24:
        
        config = random.randint(1,2)
        if config == 1:
            player1 = "color"
            player2 = "dot"
        elif config == 2:
            player1 = "dot"
            player2 = "color"
       
        
    
    if board.game_counter % 2 == 0:
        
        Y,result1,result2,result3 = board.check_board(player1)
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("\n --------------")
            print(Y)
            print("Player 1 has won!")
            break
        
        print("\nAI:Player1\n")
        
        "res contains the best possible move that was raised to the current level"
        res = algo.minimax(board,2,True)
        
        if Trace == "yes":
            algo.trace(file)
            
        pos = res[1][0][0]
        o = res[1][1]
        
        print("\nAI placement position: " + str(pos) + " | & Orientation: " + str(o) +"\n")
        
        "insert the best possible move"
        board.insert(board.X,pos,o)
        
        Y,result1,result2,result3 = board.check_board(player1)
        
        board.player1_cards-=1

        
        print("\nAI Boards:\n")
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("Player 1 has won!")
            break
        
       
            
    else:
        
        Y,result1,result2,result3 = board.check_board(player2)
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("\n --------------")
            print(Y)
            print("Player 2 has won!")
            break
            
        print("Player2: " + "# of Cards left: " + str(board.player2_cards))
        user_input = (input("Enter the position and orientation: ")) # Column of choice
        user_input = user_input.split()
        o = int(user_input[0]) # Orientation
        c = user_input[1]
        r = int(user_input[2])
        if o >= 9 or o <= 0:
            print("Orientations range between 1 - 8, please try again.")
            continue
            
        check = board.insert(board.X,[row_dict[r],col_dict[c]],o) # The dictionaries map the custom input to input that fits the numpy array
        if check == "position is taken" or check == "illegal move": # User will be able to try again (game counter is not affected)
            print(check + ", please try again.")
            continue
            
        Y,result1,result2,result3 = board.check_board(player2)

        board.player2_cards-=1
        
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 == "win" or result3 == "win":
            print("Player 2 has won!")
            break
       
        
    board.game_counter+=1 # variable to help alternate between player 1 and 2
    board.total_cards-=1
    #board.error_count_ai = 0
                  
if Trace == "yes":
    if result1 == "win" or result2 =="win" or result3 == "win":  
        file.close()
        
        
if(board.total_cards == 0 and result1 == "no win" and result2 == "no win" and result3 == "no win"):
    board.game_counter = 0
    print("\n")
    print("Recycling mode is on :")
    print("\nNote that recycling input follows this format [primary cell you want to move],[orienation of the card],[new cell of the card]")
    print("\n You won't need to specify the second cell of the card you're trying to move because the card orientation already determined it's position.")
    
    while board.recycle_counter > 0:
        
        odd = [1,3,5,7]
        even = [2,4,6,8]
        
        if board.game_counter % 2 == 0:
            
            Y,result1,result2,result3 = board.check_board(player2)
            if result1 == "win" or result2 =="win" or result3 == "win":
                print("\n --------------")
                print(Y)
                print("Player 2 has won!")
                break
            
            print("Player2, please insert the details of your recycling move")
            user_input = (input("Enter the card position, orienation, and new card positon. EX: A 1 2 D 1 ")) # Column of choice
            user_input = user_input.split()
            c1 = user_input[0]
            r1 = int(user_input[1])
            o = int(user_input[2])
            c2 = user_input[3]
            r2 = int(user_input[4])
            
            if o >= 9 or o <= 0:
                print("Orientations range between 1 - 8, please try again.")
                continue
                
            card = board.X[row_dict[r1],col_dict[c1]]
            
            if type(card) == type(None):
                print("There is no card placed here, please try again.")
                continue
                
            if card.orientation == 1 or card.orientation == 3 or card.orientation == 5 or card.orientation == 7:
                if type(board.X[card.primary_position[0]-1,card.primary_position[1]]) is not type(None):
                    print("hole error, please try again")
                    continue
            else:
                if card.secondary_position[0]-1 > 0:
                    if type(board.X[card.secondary_position[0]-1,card.secondary_position[1]]) is not type(None):
                        print("hole error, please try again")
                        continue
                
            if type(board.X[row_dict[r2],col_dict[c2]]) is not type(None) and [r2,c2] != [r1, c1] and card.secondary_position != [row_dict[r2],col_dict[c2]]:
                print("position isn't available for placement, please try again")
                continue
                
            if [r1,c1] == [r2,c2] and card.orientation == o:
                print("You can't not make a move, please try again.")
                continue
            
            
            "CASE: if the card we want to move is even, and the destionation position overlaps on the secondary position(upper part of the card)"    
            if(card != board.recycled_card):
                board.recycled_card = None
                if [row_dict[r2],col_dict[c2]] == card.secondary_position and o in even:
                    "Empty positions of moved card"
                    board.X[card.primary_position[0],card.primary_position[1]] = None
                    board.X[card.secondary_position[0],card.secondary_position[1]] = None
                    "Insert card with new orientation"
                    board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    "Create the string representation of the matrix"
                    
                    Y,result1,result2,result3 = board.check_board(player2)
                        
                    "Have the new card be marked as recycled"
                    board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                    
                    board.plot_map(Y)
                    print(Y)
                    if result1 == "win" or result2 == "win" or result3 == "win":
                        print("Player 2 has won!")
                        break
                else:
                    "CASE: regular check"
                    check_var = board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    if (check_var != "illegal move" and check_var != "position is taken"):
                        "Empty positions of moved card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        
                        Y,result1,result2,result3 = board.check_board(player2)
                            
                        "Have the new card be marked as recycled"    
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 2 has won!")
                            break

                    elif check_var != "illegal move" and check_var == "position is taken" and [r2,c2] == [r1, c1]:
                        "Empty positions of moving card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                        
                        Y,result1,result2,result3 = board.check_board(player1)
                            
                        "Have the new card be marked as recycled"
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                       
                        board.plot_map(Y)
                        print(Y)
                        
                        #print(board.recycled_card)
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 2 has won!")
                            break
                    else:
                        print("Move is not possible, try again")
                        continue
            else:
                print("This card has been recycled")
                continue
                
        else: # next player
            
            Y,result1,result2,result3 = board.check_board(player1)
            if result1 == "win" or result2 =="win" or result3 == "win":
                print("\n --------------")
                print(Y)
                print("Player 1 has won!")
                break
            
            print("\n AI Player \n")
            res = algo.minimax_rec(board,2,True)
            r1 = res[1][0][0] # Row of card to move
            c1 = res[1][0][1] # Col of card to move
            r2 = res[1][1][0] # Row of new destination
            c2 = res[1][1][1] # Col of new destination
            o = res[1][2]     # New orientation
            
            if Trace == "yes":
                algo.trace(file)

                
            "If the card we want to move is (odd or even), remove the following positions"
            if board.X[r1,c1].orientation in odd:
                if c1 + 1 < 8: # Limiting bounds
                    board.X[r1,c1] = None
                    board.X[r1,c1+1] = None
            elif board.X[r1,c1].orientation in even:
                if r1-1 > 0:
                    board.X[r1,c1] = None
                    board.X[r1-1,c1] = None
                    
            "Insert moved card into new location"  
            board.insert(board.X,[r2,c2],o)
            card = board.X[r2,c2]
            "Have the new card marked as recycled"
            board.recycled_card = card
            
            print("\n AI move: " + "[" +str(r1)+ "," +str(c1)+ "]" + "------>" + "[" +str(r2)+ "," +str(c2)+  "]" + " with orientation " + str(o))
            
            Y,result1,result2,result3 = board.check_board(player1)
            
            board.plot_map(Y)
            print(Y)
            
            if result1 == "win" or result2 == "win" or result3 == "win":
                print("Player 1 has won!")
                break
                    
        
        board.game_counter+=1
        board.recycle_counter-=1
        
if result1 != "win" and result2 != "win" and result3 != "win":
    print("The game ends in draw.")

if Trace == "yes":
    file.close()

# Match2 - Opponents AI is Player1

In [ ]:
# Initialize board
board = Board()
col_dict = board.col_mappings() # Map from custom input to numpy input
row_dict = board.row_mappings() # MAo from custom input to numpy input
counter = 0
algo = AI()
H = Informed_Heuristic()

Trace = str(input("Please enter 'yes' if you'd like to generate a trace file for the minimax function.\n"))
if Trace == "yes":
    file = open("tracemm1.txt","w")

print("\n input format example: 1 A 1 or Orientation,Column,Row\n")
while board.total_cards != 0:
    
    if board.total_cards == 24:
        player1 = str(input("Player1, please enter what you will play (color or dot)"))
        if player1 == "color":
            player2 = "dot"
        else:
            player2 = "color"
        
    
    if board.game_counter % 2 == 0:
        
        Y,result1,result2,result3 = board.check_board(player1)
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("\n --------------")
            print(Y)
            print("Player 1 has won!")
            break
            
        print("Player1: " + "# of Cards left: " + str(board.player1_cards))
        user_input = (input("Enter the position and orientation: ")) # Column of choice
        user_input = user_input.split()
        o = int(user_input[0]) # Orientation
        c = user_input[1]
        r = int(user_input[2])
        if o >= 9 or o <= 0:
            print("Orientations range between 1 - 8, please try again.")
            continue
            
        check = board.insert(board.X,[row_dict[r],col_dict[c]],o) # The dictionaries map the custom input to input that fits the numpy array
        if check == "position is taken" or check == "illegal move": # User will be able to try again (game counter is not affected)
            print(check + ", please try again.")
            continue
            
        Y,result1,result2,result3 = board.check_board(player1)

        board.player1_cards-=1
        
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 == "win" or result3 == "win":
            print("Player 1 has won!")
            break
            
    else:
        
        Y,result1,result2,result3 = board.check_board(player2)
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("\n --------------")
            print(Y)
            print("Player 2 has won!")
            break
        
        "res contains the best possible move that was raised to the current level"
        res = algo.minimax(board,2,True)
        
        if Trace == "yes":
            algo.trace(file)
            
        pos = res[1][0][0]
        o = res[1][1]
        
        print("\nAI placement position: " + str(pos) + " | & Orientation: " + str(o) +"\n")
        
        "insert the best possible move"
        board.insert(board.X,pos,o)
        
        Y,result1,result2,result3 = board.check_board(player2)
      
        board.player2_cards-=1
        
        print("\nAI Boards:\n")
        board.plot_map(Y)
        print(Y)
        
        if result1 == "win" or result2 =="win" or result3 == "win":
            print("Player 2 has won!")
            break
    
    
    board.game_counter+=1 # variable to help alternate between player 1 and 2
    board.total_cards-=1
    #board.error_count_ai = 0
                  
if Trace == "yes":
    if result1 == "win" or result2 =="win" or result3 == "win":  
        file.close()
        
        
if(board.total_cards == 0 and result1 == "no win" and result2 == "no win" and result3 == "no win"):
    board.game_counter = 0
    print("\n")
    print("Recycling mode is on :")
    print("\nNote that recycling input follows this format [primary cell you want to move],[orienation of the card],[new cell of the card]")
    print("\n You won't need to specify the second cell of the card you're trying to move because the card orientation already determined it's position.")
    
    while board.recycle_counter > 0:
        
        odd = [1,3,5,7]
        even = [2,4,6,8]
        
        if board.game_counter % 2 == 0:
            
            Y,result1,result2,result3 = board.check_board(player1)
            if result1 == "win" or result2 =="win" or result3 == "win":
                print("\n --------------")
                print(Y)
                print("Player 1 has won!")
                break
            
            print("Player1, please insert the details of your recycling move")
            user_input = (input("Enter the card position, orienation, and new card positon. EX: A 1 2 D 1 ")) # Column of choice
            user_input = user_input.split()
            c1 = user_input[0]
            r1 = int(user_input[1])
            o = int(user_input[2])
            c2 = user_input[3]
            r2 = int(user_input[4])
            
            if o >= 9 or o <= 0:
                print("Orientations range between 1 - 8, please try again.")
                continue
                
            card = board.X[row_dict[r1],col_dict[c1]]
            
            if type(card) == type(None):
                print("There is no card placed here, please try again.")
                continue
                
            if card.orientation == 1 or card.orientation == 3 or card.orientation == 5 or card.orientation == 7:
                if type(board.X[card.primary_position[0]-1,card.primary_position[1]]) is not type(None):
                    print("hole error, please try again")
                    continue
            else:
                if card.secondary_position[0]-1 > 0:
                    if type(board.X[card.secondary_position[0]-1,card.secondary_position[1]]) is not type(None):
                        print("hole error, please try again")
                        continue
                
            if type(board.X[row_dict[r2],col_dict[c2]]) is not type(None) and [r2,c2] != [r1, c1] and card.secondary_position != [row_dict[r2],col_dict[c2]]:
                print("position isn't available for placement, please try again")
                continue
                
            if [r1,c1] == [r2,c2] and card.orientation == o:
                print("You can't not make a move, please try again.")
                continue
            
            
            "CASE: if the card we want to move is even, and the destionation position overlaps on the secondary position(upper part of the card)"    
            if(card != board.recycled_card):
                board.recycled_card = None
                if [row_dict[r2],col_dict[c2]] == card.secondary_position and o in even:
                    "Empty positions of moved card"
                    board.X[card.primary_position[0],card.primary_position[1]] = None
                    board.X[card.secondary_position[0],card.secondary_position[1]] = None
                    "Insert card with new orientation"
                    board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    "Create the string representation of the matrix"
                    
                    Y,result1,result2,result3 = board.check_board(player1)
       
                        
                    "Have the new card be marked as recycled"
                    board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                    
                    board.plot_map(Y)
                    print(Y)
                    if result1 == "win" or result2 == "win" or result3 == "win":
                        print("Player 1 has won!")
                        break
                else:
                    "CASE: regular check"
                    check_var = board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                    if (check_var != "illegal move" and check_var != "position is taken"):
                        "Empty positions of moved card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        
                        Y,result1,result2,result3 = board.check_board(player1)
                            
                        "Have the new card be marked as recycled"    
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                        
                        board.plot_map(Y)
                        print(Y)
                        
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break

                    elif check_var != "illegal move" and check_var == "position is taken" and [r2,c2] == [r1, c1]:
                        "Empty positions of moving card"
                        board.X[card.primary_position[0],card.primary_position[1]] = None
                        board.X[card.secondary_position[0],card.secondary_position[1]] = None
                        board.insert(board.X,[row_dict[r2],col_dict[c2]],o)
                        
                        Y,result1,result2,result3 = board.check_board(player1)
        
                        "Have the new card be marked as recycled"
                        board.recycled_card = board.X[row_dict[r2],col_dict[c2]]
                       
                        board.plot_map(Y)
                        print(Y)
                        
                        #print(board.recycled_card)
                        if result1 == "win" or result2 == "win" or result3 == "win":
                            print("Player 1 has won!")
                            break
                    else:
                        print("Move is not possible, try again")
                        continue
            else:
                print("This card has been recycled")
                continue
                
        else: # next player
            
            Y,result1,result2,result3 = board.check_board(player2)
            if result1 == "win" or result2 =="win" or result3 == "win":
                print("\n --------------")
                print(Y)
                print("Player 2 has won!")
                break
            
            print("\n AI Player \n")
            res = algo.minimax_rec(board,2,True)
            r1 = res[1][0][0] # Row of card to move
            c1 = res[1][0][1] # Col of card to move
            r2 = res[1][1][0] # Row of new destination
            c2 = res[1][1][1] # Col of new destination
            o = res[1][2]     # New orientation
            
            if Trace == "yes":
                algo.trace(file)
                
            "If the card we want to move is (odd or even), remove the following positions"
            if board.X[r1,c1].orientation in odd:
                if c1 + 1 < 8: # Limiting bounds
                    board.X[r1,c1] = None
                    board.X[r1,c1+1] = None
            elif board.X[r1,c1].orientation in even:
                if r1-1 > 0:
                    board.X[r1,c1] = None
                    board.X[r1-1,c1] = None
                    
            "Insert moved card into new location"  
            board.insert(board.X,[r2,c2],o)
            card = board.X[r2,c2]
            "Have the new card marked as recycled"
            board.recycled_card = card
            
            print("\n AI move: " + "[" +str(r1)+ "," +str(c1)+ "]" + "------>" + "[" +str(r2)+ "," +str(c2)+  "]" + " with orientation " + str(o))
            
            Y,result1,result2,result3 = board.check_board(player2)
        
            
            board.plot_map(Y)
            print(Y)
            
            if result1 == "win" or result2 == "win" or result3 == "win":
                print("Player 2 has won!")
                break
                    
        
        board.game_counter+=1
        board.recycle_counter-=1
        
if result1 != "win" and result2 != "win" and result3 != "win":
    print("The game ends in draw.")

if Trace == "yes":
    file.close()